# Solution Task 4 (Week 04)<a href="#Solution-Task-4-(Week-04)" class="anchor-link">¶</a>

### Libraries and settings<a href="#Libraries-and-settings" class="anchor-link">¶</a>

In \[ \]:

    import os
    import json
    import pandas as pd
    import sqlite3
    from pandas import json_normalize

    print(os.getcwd())

    /workspaces/data_analytics/Week_02

## Additional SQL-queries<a href="#Additional-SQL-queries" class="anchor-link">¶</a>

### 4c) Filter all apartments with \>= 4.0 rooms and where the area is \>= 100m2<a
href="#4c)-Filter-all-apartments-with-%3E=-4.0-rooms-and-where-the-area-is-%3E=-100m2"
class="anchor-link">¶</a>

In \[ \]:

    # Open a connection to the data base 'apartment_database.db'
    conn = sqlite3.connect('apartment_database.db') 

    # Create cursor object
    cursor = conn.cursor()

    # Query the SQL-table
    cursor.execute('''SELECT *
                   FROM apartments_table
                   WHERE rooms >=4.0
                   AND area >= 100''')

    df = pd.DataFrame(cursor.fetchall(), 
                      columns=['OrderId','Address','Rooms','Area','Price'])    

    # Close connection
    conn.close()

    # Show results
    df.head()

Out\[ \]:

|     | OrderId       | Address                               | Rooms | Area  | Price  |
|-----|---------------|---------------------------------------|-------|-------|--------|
| 0   | 1693993818-1  | Am Eulachpark 25, 8404 Winterthur, ZH | 6.5   | 143.0 | 3017.0 |
| 1   | 1693993818-3  | 8400 Winterthur, ZH                   | 4.5   | 117.0 | 3782.0 |
| 2   | 1693993818-10 | Marktgasse 17, 8400 Winterthur, ZH    | 4.5   | 160.0 | 3200.0 |
| 3   | 1693993818-12 | Hobelwerkweg 39, 8404 Winterthur, ZH  | 4.5   | 103.0 | 2470.0 |
| 4   | 1693993818-20 | 8400 Winterthur, ZH                   | 4.0   | 100.0 | 3125.0 |

### 4d) Write an SQL-query to calculate the average price per room size<a
href="#4d)-Write-an-SQL-query-to-calculate-the-average-price-per-room-size"
class="anchor-link">¶</a>

In \[ \]:

    # Open a connection to the data base 'apartment_database.db'
    conn = sqlite3.connect('apartment_database.db') 

    # Create cursor object
    cursor = conn.cursor()

    # Query the SQL-table
    cursor.execute('''SELECT rooms, AVG(price)    
                   FROM apartments_table    
                   GROUP BY rooms''')

    df = pd.DataFrame(cursor.fetchall(), 
                      columns=['Rooms','Avg_Price'])    

    # Close connection
    conn.close()

    # Show results
    df

Out\[ \]:

|     | Rooms | Avg_Price   |
|-----|-------|-------------|
| 0   | NaN   | 1880.000000 |
| 1   | 1.0   | 1398.333333 |
| 2   | 1.5   | 1875.000000 |
| 3   | 2.0   | 1566.666667 |
| 4   | 2.5   | 1877.923077 |
| 5   | 3.0   | 1515.375000 |
| 6   | 3.5   | 2182.125000 |
| 7   | 4.0   | 2732.500000 |
| 8   | 4.5   | 2624.103448 |
| 9   | 5.5   | 3023.166667 |
| 10  | 6.5   | 3017.000000 |

### 4e) Write an SQL-query to calculate the average area per room size<a
href="#4e)-Write-an-SQL-query-to-calculate-the-average-area-per-room-size"
class="anchor-link">¶</a>

In \[ \]:

    # Open a connection to the data base 'apartment_database.db'
    conn = sqlite3.connect('apartment_database.db') 

    # Create cursor object
    cursor = conn.cursor()

    # Query the SQL-table
    cursor.execute('''SELECT rooms, AVG(area)    
                   FROM apartments_table    
                   GROUP BY rooms''')

    df = pd.DataFrame(cursor.fetchall(), 
                      columns=['Rooms','Avg_Area'])    

    # Close connection
    conn.close()

    # Show results
    df

Out\[ \]:

|     | Rooms | Avg_Area   |
|-----|-------|------------|
| 0   | NaN   | NaN        |
| 1   | 1.0   | 57.500000  |
| 2   | 1.5   | 34.000000  |
| 3   | 2.0   | 53.400000  |
| 4   | 2.5   | 70.040000  |
| 5   | 3.0   | 64.571429  |
| 6   | 3.5   | 94.043478  |
| 7   | 4.0   | 96.000000  |
| 8   | 4.5   | 110.071429 |
| 9   | 5.5   | 152.000000 |
| 10  | 6.5   | 143.000000 |

### 4f) Create a data base for the supermarket data<a href="#4f)-Create-a-data-base-for-the-supermarket-data"
class="anchor-link">¶</a>

In \[ \]:

    # Create data base
    conn = sqlite3.connect('supermarkets_database.db') 
    cursor = conn.cursor()

    # Load the JSON data from file
    with open('supermarkets.json') as json_file:
        data = json.load(json_file)

    # Use json_normalize to flatten the JSON and create a DataFrame
    df = pd.json_normalize(data)

    # Select columns
    df = df[["id", "lat", "lon", "tags.brand", "tags.shop", 
               "tags.addr:city", "tags.addr:street", 
               "tags.addr:housenumber", "tags.addr:postcode"]]

    # Rename columns
    df = df.rename(columns={"tags.brand":"brand", 
                            "tags.shop":"shop", 
                            "tags.addr:city":"city", 
                            "tags.addr:street":"street", 
                            "tags.addr:housenumber":"housenumber", 
                            "tags.addr:postcode":"postcode"})

    # Create db table
    cursor.execute('''CREATE TABLE IF NOT EXISTS supermarkets_table (id INT(20),
                                                                     lat DECIMAL(8,2),
                                                                     lon DECIMAL(8,2),
                                                                     brand VARCHAR(50),
                                                                     shop VARCHAR(50),
                                                                     city VARCHAR(100),
                                                                     street VARCHAR(200),
                                                                     housenumber VARCHAR(50),
                                                                     postcode INT(4))''')
    # Confirm changes to the table
    conn.commit()

    # Write data to table
    df.to_sql(name = 'supermarkets_table',
              con = conn,
              index = False,
              if_exists = 'replace')

Out\[ \]:

    3392

### 4g) Write an SQL-query to filter all supermarkets in the city of Winterthur<a
href="#4g)-Write-an-SQL-query-to-filter-all-supermarkets-in-the-city-of-Winterthur"
class="anchor-link">¶</a>

In \[ \]:

    # Open a connection to the data base 'apartment_database.db'
    conn = sqlite3.connect('supermarkets_database.db') 

    # List tables in supermarket_database.db
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print('Tables in supermarkets_database.db:\n', tables)

    # List column names in supermarket_table
    cursor = conn.execute('SELECT * FROM supermarkets_table')
    colnames = list(map(lambda x: x[0], cursor.description))
    print('\nColnames in supermarkets_table:\n', colnames)

    # Query the SQL-table
    print('\nSQL query')
    cursor.execute('''SELECT * FROM supermarkets_table WHERE city = "Winterthur"''')
    df = pd.DataFrame(cursor.fetchall(), 
                      columns=['id', 'lat', 'lon', 'brand', 'shop', 'city', 'street', 'housenumber', 'postcode'])

    df.head()

    Tables in supermarkets_database.db:
     [('supermarkets_table',)]

    Colnames in supermarkets_table:
     ['id', 'lat', 'lon', 'brand', 'shop', 'city', 'street', 'housenumber', 'postcode']

    SQL query

Out\[ \]:

|     | id        | lat       | lon      | brand  | shop        | city       | street              | housenumber | postcode |
|-----|-----------|-----------|----------|--------|-------------|------------|---------------------|-------------|----------|
| 0   | 70656488  | 47.491874 | 8.706448 | Migros | supermarket | Winterthur | Zürcherstrasse      | 102         | 8406     |
| 1   | 246533376 | 47.518953 | 8.720071 | Migros | supermarket | Winterthur | Schaffhauserstrasse | 152         | 8400     |
| 2   | 548919417 | 47.506863 | 8.712208 | Migros | supermarket | Winterthur | Wülflingerstrasse   | 71          | 8400     |
| 3   | 677194765 | 47.503208 | 8.752131 | None   | supermarket | Winterthur | Frauenfelderstrasse | 69          | 8404     |
| 4   | 706203439 | 47.500848 | 8.725938 | None   | supermarket | Winterthur | Bankstrasse         | 8/12        | 8400     |